In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parallel Processing 

**1**. (30 points) Accelerating network bound procedures.

Python code to show PNG images available on the URL `http://people.duke.edu/~ccc14/misc/` is provided.

- Write a funciton `download_one(url, path)` that downloads one image given a `url` and saves the image to `path` (5 points)
    
For the exercises below, save the image file to the local directory using the same fileanme as on `http://people.duke.edu/~ccc14/misc/`. You can just use the `%%time` magic to time the downloads.
 
- Write a for loop to download all images and time how long it takes (5 points)
- Write a function that uses `concurrent.futures` and a thread pool with 4 threads to download all images and time how long it takes (5 points)
- Write a function that uses `multiprocessing` and a process pool with 4 processes to download all images and time how long it takes (5 points)
- Write a function that uses `ipyparallel` and a process pool with 4 processes to download all images and time how long it takes (10 points)

In [3]:
import requests
from bs4 import BeautifulSoup

def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') 
            if node.get('href').endswith(ext)]

site = 'http://people.duke.edu/~ccc14/misc/'
ext = 'png'
for i, file in enumerate(listFD(site, ext)):
    print(file)

http://people.duke.edu/~ccc14/misc/250px-002Ivysaur.png
http://people.duke.edu/~ccc14/misc/250px-003Venusaur.png
http://people.duke.edu/~ccc14/misc/250px-004Charmander.png
http://people.duke.edu/~ccc14/misc/250px-005Charmeleon.png
http://people.duke.edu/~ccc14/misc/250px-006Charizard.png
http://people.duke.edu/~ccc14/misc/250px-008Wartortle.png
http://people.duke.edu/~ccc14/misc/250px-009Blastoise.png
http://people.duke.edu/~ccc14/misc/250px-012Butterfree.png
http://people.duke.edu/~ccc14/misc/250px-015Beedrill.png
http://people.duke.edu/~ccc14/misc/250px-016Pidgey.png
http://people.duke.edu/~ccc14/misc/250px-017Pidgeotto.png
http://people.duke.edu/~ccc14/misc/250px-018Pidgeot.png
http://people.duke.edu/~ccc14/misc/250px-019Rattata.png
http://people.duke.edu/~ccc14/misc/250px-020Raticate.png
http://people.duke.edu/~ccc14/misc/250px-021Spearow.png
http://people.duke.edu/~ccc14/misc/250px-022Fearow.png
http://people.duke.edu/~ccc14/misc/250px-023Ekans.png
http://people.duke.edu/~ccc14/mi

### Function to download a single image at URL to PATH

### Using for loop

Try to download one

In [4]:
pwd

'/home/jovyan/work/sta-663-2018/labs'

In [5]:
from PIL import Image
import os
os.path

<module 'posixpath' from '/opt/conda/lib/python3.6/posixpath.py'>

In [6]:
url = listFD(site, ext)[1]
response = requests.get(url)
url

'http://people.duke.edu/~ccc14/misc/250px-003Venusaur.png'

In [7]:
file_name=url.split('/')[-1]
#file_name="/lab11/"+file_name

In [8]:
if response.status_code == 200:
    img = response.raw.read()
    with open(file_name, 'wb') as f:
        f.write(img)

The function

In [9]:
def download_one(url, path):
    """
    Function to download a single image at URL to PATH
    """
    response = requests.get(url)
    if response.status_code == 200:
        img = response.raw.read()
        with open(path, 'wb') as f:
            f.write(img)

In [10]:
%%time

for url in listFD(site, ext):
    paths = url.split('/')[-1]
    download_one(url, paths)

CPU times: user 376 ms, sys: 100 ms, total: 476 ms
Wall time: 8.39 s


### Using a thread pool

In [11]:
from concurrent.futures import ThreadPoolExecutor

In [12]:
len(listFD(site, ext))

105

In [13]:
paths=[listFD(site, ext)[i].split('/')[-1] for i in range(len(listFD(site, ext)))]

In [14]:
urls='http://people.duke.edu/~ccc14/misc/'

In [15]:
urls+paths[1]

'http://people.duke.edu/~ccc14/misc/250px-003Venusaur.png'

In [16]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    pool.map(lambda x: download_one(urls+x, x), paths)

CPU times: user 344 ms, sys: 104 ms, total: 448 ms
Wall time: 2.11 s


### Using a process pool

In [17]:
from multiprocessing import Pool

In [18]:
files=[(urls+paths[i],paths[i]) for i in range(len(paths))]

In [19]:
%%time

with Pool(processes=4) as pool:
    pool.starmap(download_one, files)

CPU times: user 12 ms, sys: 20 ms, total: 32 ms
Wall time: 2.23 s


### Using `ipyparallel`

In [27]:
from ipyparallel import Client
rc = Client()
dv = rc[:]

In [24]:
@dv.parallel(block = True)
def download_one2(url, path):
    response = requests.get(url)
    if response.status_code == 200:
        img = response.raw.read()
        with open(path, 'wb') as f:
            f.write(img)

In [25]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    pool.map(lambda x: download_one2(urls+x, x), paths)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 13.4 ms
